# Multi-modelo e Multi-ordem: Uma proposta de aplicação em séries temporais.

Diversos métodos podem ser empregados na predição de variáveis descritas como séries temporais. Dentre esses métodos, destacam-se as técnicas baseadas em equações em diferenças, como: Arma, Arima, Sarima, Var, Vec-Var e etc. Comumente a estimação desses modelos envolve as seguintes etapas: teste de estacionariedade das séries empregadas no modelo, teste de cointegração (para o caso de modelos multivariados cuja ordem de integração das séries seja superior a 0), seleção da ordem do modelo e teste de desempenho do modelo pós estimação. &nbsp;

A etapa de seleção da ordem do modelo costuma ser realizada com critérios de informação (AIC, BIC e HQ). O modelo é estimado com diferentes ordens e, a partir dos critérios de informação, seleciona-se a ordem que melhor se adequar ao critério de informação escolhido. Entretanto, é comum que determinadas séries apresentem períodos de maior volatilidade, bem como períodos de maior estabilidade. Ainda é comum que algumas séries apresentem inversões de tendência ao longo do tempo. Logo, é factível que diferentes momentos de uma determinada série temporal apresentem diferentes ordens ótimas para a obtenção da melhor predição. &nbsp;

Ademais, é comum a comparação de desempenho entre diferentes modelos na predição de uma determinada série. Assim, escolhe-se o modelo com melhor desempenho para ser empregado na predição. Entretanto, assim como na seleção da ordem, os diferentes momentos desempenhados por uma determinada série temporal podem ser melhor previstos por modelos diferentes. &nbsp;

Desse modo, propõe-se uma metodologia que empregue diferentes ordens e diferentes modelos conforme os movimentos de uma determinada variável (variável de regime). &nbsp;

Para exemplificar a adoção de tal metodologia, utilizamos dois modelos: Arima e Var. Para o Arima, utilizamos às ordens: Ordem *p* de 1 a 2 e ordem *q* de 0 a 1. Para o Var, utilizamos a ordem *p* de 1 a 3. &nbsp;

Quanto as séries, utilizamos: Preço da Soja, Preço do Milho e Taxa de câmbio (R$/U$$). As séries de preço foram obtidas no Centro de Estudos Avançados em Economia Aplicada (CEPEA). &nbsp;

O objetivo de toda a modelagem foi a previsão do Preço da Soja.
 

In [7]:
import pandas as pd
import numpy as np
import statsmodels.tsa.api as stts
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
import plotly.express as px
import itertools
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.api import VAR

In [8]:
### Importando os arquivos em .csv
soy_p = pd.read_csv('C:/Users/User/Documents/fabio/port/alg_multi_model/soja.csv', sep = ';')
milho = pd.read_csv('C:/Users/User/Documents/fabio/port/alg_multi_model/milho.csv', sep = ';')
tx = pd.read_csv('C:/Users/User/Documents/fabio/port/alg_multi_model/tx.csv', sep = ';')

In [9]:
## Transformando a coluna data em "datetime"
soy_p['data'] = pd.to_datetime(soy_p['data'], format= '%d/%m/%Y')
milho['data'] = pd.to_datetime(milho['data'], format= '%d/%m/%Y')
tx['data'] = pd.to_datetime(tx['data'], format= '%d/%m/%Y')

### Merge nos datasets
soy_p = soy_p.merge(milho, left_on='data',right_on='data')
soy_p = soy_p.merge(tx, left_on='data',right_on='data')

##### Modificando o index para a coluna de 'data' e estabelecendo a frequência diária.
soy_p = soy_p.set_index('data')
soy_p

,p_soja,milho_p,tx
data,,,
2010-01-04,42.00,20.39,1.7232
2010-01-05,42.00,20.40,1.7219
2010-01-06,41.50,20.25,1.7329
2010-01-07,42.00,20.28,1.7405
2010-01-08,42.00,20.18,1.7382
...,...,...,...
2022-02-14,191.08,95.98,5.2100
2022-02-15,189.54,96.55,5.1875
2022-02-16,189.59,96.53,5.1624


In [10]:
### Verificando se o data set possui "na"
soy_p.isnull().values.any()

False

In [11]:
### Evolução das séries ao longo dos períodos
fig = px.line(soy_p)
fig.show()

In [12]:
# Convertendo o 'index' em 'datetime'
soy_p.index = pd.DatetimeIndex(soy_p.index).to_period('d')

## Teste de Raiz Unitária

Para checar a presença de raiz unitária nas séries, utilizamos o teste ADF. O teste ADF pode ser descrito como um teste sequencial entre três especificações: M3 (com tendência e drift); M2 (drift) e M1 (sem termos adicionais). Justifica-se a existência das três especificações pela distribuição da estatística tau empregada no teste. O uso dos termos de tendência e drift nas equações modifica a distribuição tau, de modo que é necessário o emprego da especificação condizente a série em questão. Caso contrário, pode-se rejeitar (ou não rejeitar) erroneamente a hipótese nula de presença da raiz unitária. Desse modo, empregamos as 3 especificações em todas a séries que compõem a modelagem. Uma vez que o pacote *statsmodels* não apresenta as informações necessárias para que se calcule a estatística *F* (o teste *F* é utilizado para a seleção da especificação correta) entre as especificações, optamos por realizar a estimação do teste sem a estatística *F*. Se as 3 especificações não apresentarem diferenças a despeito da presença de raiz unitária, não é necessário que se determine qual especificação é a correta.

In [13]:
# Função para o teste adf na série em nível e na diferença.

def adf(ts):
    ts_diff = np.diff(ts)
    m3 = adfuller(ts,regression='ct')
    m2 = adfuller(ts,regression='c')
    m1 = adfuller(ts,regression='n')
    m3_diff = adfuller(ts_diff,regression='ct')
    m2_diff = adfuller(ts_diff,regression='c')
    m1_diff = adfuller(ts_diff,regression='n')
    print('Series:',ts.name)
    print('Level: ','M3: lags: ',m3[2],'; Stat.:',m3[0],' (p-value:',m3[1],')',' M2: lags: ',m2[2],'; Stat.:',m2[0],' (p-value:',m2[1],')',
        ' M1: lags: ',m1[2],'; Stat.:',m1[0],' (p-value:',m1[1],')',sep='')
    print('Diff: ','M3: lags: ',m3_diff[2],'; Stat.:',m3_diff[0],' (p-value:',m3_diff[1],')',' M2: lags: ',m2_diff[2],'; Stat.:',m2_diff[0],' (p-value:',m2_diff[1],')',
        ' M1: lags: ',m1_diff[2],'; Stat.:',m1_diff[0],' (p-value:',m1_diff[1],')',sep='')
    

soy_p.apply(adf, axis=0)

Series: p_soja
Level: M3: lags: 24; Stat.:-0.2707402732834306 (p-value:0.9901641727455346) M2: lags: 24; Stat.:1.175306987973386 (p-value:0.995810757764797) M1: lags: 24; Stat.:2.4654735924566413 (p-value:0.9978578228737468)
Diff: M3: lags: 23; Stat.:-10.30132746849073 (p-value:5.409446068768283e-16) M2: lags: 23; Stat.:-10.165949280774939 (p-value:7.29856372535702e-18) M1: lags: 23; Stat.:-9.925832130753898 (p-value:1.453779090729691e-17)
Series: milho_p
Level: M3: lags: 5; Stat.:-1.3614871221137133 (p-value:0.8718669420638709) M2: lags: 5; Stat.:0.004857531151828204 (p-value:0.9589319616726603) M1: lags: 5; Stat.:1.277569917782785 (p-value:0.9484143369776034)
Diff: M3: lags: 4; Stat.:-15.364348166181866 (p-value:1.5363103793503854e-22) M2: lags: 4; Stat.:-15.315134202604028 (p-value:4.1355879866849085e-28) M1: lags: 4; Stat.:-15.245803113708389 (p-value:6.069260505077936e-27)
Series: tx
Level: M3: lags: 25; Stat.:-2.8670114824654855 (p-value:0.17342080844858915) M2: lags: 25; Stat.:-

p_soja     None
milho_p    None
tx         None
dtype: object

De acordo com o resultado do teste, todas a séries não são estacionárias em nível, mas se tornam estacionárias na primeira diferença. 

## Funções de seleção multi-ordem

As funções de seleção de multi-ordem têm como objetivo armazenar o *MAPE* de 7 dias de predição para cada uma das ordens testadas, simulando o processo de predição diário. Em outras palavras, a função adapta os princípios descritos na validação cruzada em séries temporais da seguinte forma: Inicia-se retirando *n* dias da série temporal. Em seguida, estima-se o modelo com todas as ordens candidatas, bem como realiza-se a predição com cada uma dessas ordens (no caso, realizamos a predição para 7 dias). Os resultados da predição de cada ordem são comparados, via *MAPE* com as observações reais (no caso, essas observações pertencem ao *n* retirados da série temporal) e armazenados. Em seguida, uma das *n* observações que foram retiradas da série temporal é adicionada no conjunto de dados e o processo de estimação e predição se repete. Esse processo perdura até que todos os *n* dias que foram retirados retornem a série temporal.  

In [14]:
#### Função para o Arima
def cv_ts_arima(ts,p,q,int,day_out):
    
    ##### order
    order_arima = list(itertools.product(list(range(1,p+1)),[int],list(range(0,q+1))))
    ##### store
    store_date = []
    store_order = []
    store_mape = []

    for i in sorted(range(7,day_out+1),reverse=True):
        ts_in = ts.iloc[range(0,len(ts)-i)]
        ts_out = ts.iloc[range(len(ts)-i,len(ts)-i+7)]

        for j in range(0,len(order_arima)):
            arima = stts.statespace.SARIMAX(ts_in,trend = 'c', order = order_arima[j])
            arima_est = arima.fit(disp=False)
            predict = arima_est.get_forecast(steps= 7)
            predict = predict.summary_frame()
            mape = mean_absolute_percentage_error(ts_out, predict['mean'])
            store_date.append(ts_in.tail(1).index[0])
            store_order.append('Arima '+str(order_arima[j]))
            store_mape.append(mape)

    store = pd.DataFrame({'pred_data':store_date,'order':store_order,'mape':store_mape})
    return store

#### Sempre: day_out>7
arima_cv_store = cv_ts_arima(soy_p['p_soja'],2,1,1,15)
arima_cv_store.tail(5)

,pred_data,order,mape
31,2022-02-01,"Arima (2, 1, 1)",0.022241
32,2022-02-09,"Arima (1, 1, 0)",0.023832
33,2022-02-09,"Arima (1, 1, 1)",0.029439
34,2022-02-09,"Arima (2, 1, 0)",0.026506
35,2022-02-09,"Arima (2, 1, 1)",0.029445


In [15]:
#### Função para o Var
def cv_ts_var(ts,p,int,day_out):
    
    ##### order
    order_var = list(range(1,p+1))
    ##### store
    store_date = []
    store_order = []
    store_mape = []
    

    for i in sorted(range(7,day_out+1),reverse=True):
        if int == 1 :
            ts_diff = ts.diff().dropna()
            ts_in = ts_diff.iloc[range(0,len(ts_diff)-i)]
        else:
            ts_in = ts.iloc[range(0,len(ts)-i)]
        
        ts_out = ts.iloc[range(len(ts)-i,len(ts)-i+7)]

        for j in range(0,len(order_var)):
            varr = VAR(ts_in)
            var_est = varr.fit(order_var[j])
            lag_est = var_est.k_ar
            predict = var_est.forecast(y=ts_in.values[-lag_est:], steps= 7)
            predict = pd.DataFrame(predict)
            #### Arrumando a projeção
            store_predict = []
            store_predict.append(ts[-1:]['p_soja'].values+predict[0][0])

            for k in range(1,7):
                store_predict.append(store_predict[k-1]+predict[0][k])
            
            mape = mean_absolute_percentage_error(ts_out['p_soja'].values, store_predict)
            store_date.append(ts_in.tail(1).index[0])
            store_order.append('Var ('+ str(order_var[j]) +')')
            store_mape.append(mape)

    store = pd.DataFrame({'pred_data':store_date,'order':store_order,'mape':store_mape})
    return store

#### always day_out>7
var_cv_store = cv_ts_var(soy_p[['p_soja','tx','milho_p']],3,1,15)
var_cv_store.tail(5)

,pred_data,order,mape
22,2022-02-01,Var (2),0.018305
23,2022-02-01,Var (3),0.018410
24,2022-02-09,Var (1),0.017350
25,2022-02-09,Var (2),0.020868
26,2022-02-09,Var (3),0.021485


## Variável de Regime e Multi-modelo

Como variável de regime, utilizamos a variação percentual diária. Ademais, construímos três intervalos a partir do valor máximo e mínimo da variação percentual diária. Assim, pode-se delegar um modelo para cada um dos intervalos. 

In [16]:
### Criando a variável de regime
t_var = soy_p['p_soja'].diff().dropna()
t_var = t_var/soy_p['p_soja'] #crescimento em relação ao dia anterior
t_var = t_var.dropna()
t_var = pd.DataFrame({'data':t_var.index,'g_p_soja':t_var})
t_var.reset_index(drop=True,inplace=True)
t_var

#### Juntando as base de dados
models = arima_cv_store.append(var_cv_store)
models = models.merge(t_var, left_on='pred_data',right_on='data')
models = models[['pred_data','order','mape','g_p_soja']]
models



,pred_data,order,mape,g_p_soja
0,2022-01-21,"Arima (1, 1, 0)",0.016674,0.004219
1,2022-01-21,"Arima (1, 1, 1)",0.015828,0.004219
2,2022-01-21,"Arima (2, 1, 0)",0.016307,0.004219
3,2022-01-21,"Arima (2, 1, 1)",0.015832,0.004219
4,2022-01-21,Var (1),0.072041,0.004219
...,...,...,...,...
58,2022-02-09,"Arima (2, 1, 0)",0.026506,0.043207
59,2022-02-09,"Arima (2, 1, 1)",0.029445,0.043207
60,2022-02-09,Var (1),0.017350,0.043207
61,2022-02-09,Var (2),0.020868,0.043207


In [18]:
# Divisão do intervalo entre valor máximo e mínimo em 3 partes
max_min = (max(models['g_p_soja'])-min(models['g_p_soja']))/3
int_div1 = [min(models['g_p_soja']), min(models['g_p_soja']) + max_min]
int_div2 = [min(models['g_p_soja']) + max_min, min(models['g_p_soja']) + 2*max_min]
int_div3 = [min(models['g_p_soja']) + 2*max_min, max(models['g_p_soja'])]


In [23]:
#### Mape de cada especificação e modelo no primeiro intervalo
print('Intervalo 1:',int_div1)
print(models[models['g_p_soja'] < int_div1[1]].groupby(['order']).mean())

Intervalo 1: [-0.015098082433326037, 0.00433696477161602]
                     mape  g_p_soja
order                              
Arima (1, 1, 0)  0.037080 -0.002113
Arima (1, 1, 1)  0.036305 -0.002113
Arima (2, 1, 0)  0.036810 -0.002113
Arima (2, 1, 1)  0.036300 -0.002113
Var (1)          0.045819 -0.002113
Var (2)          0.046169 -0.002113
Var (3)          0.046210 -0.002113


In [20]:
#### Mape de cada especificação e modelo no segundo intervalo
print('Intervalo 2:',int_div2)
print(models[(models['g_p_soja']>=int_div2[0]) & (models['g_p_soja'] < int_div2[1])].groupby(['order']).mean())

Intervalo 2: [0.00433696477161602, 0.023772011976558077]
                     mape  g_p_soja
order                              
Arima (1, 1, 0)  0.032403  0.017919
Arima (1, 1, 1)  0.031239  0.017919
Arima (2, 1, 0)  0.032004  0.017919
Arima (2, 1, 1)  0.031259  0.017919
Var (1)          0.021684  0.017919
Var (2)          0.022265  0.017919
Var (3)          0.022335  0.017919


In [21]:
#### Mape de cada especificação e modelo no terceiro intervalo
print('Intervalo 3:',int_div3)
print(models[models['g_p_soja']>=int_div3[0]].groupby(['order']).mean())

Intervalo 3: [0.023772011976558077, 0.043207059181500126]
                     mape  g_p_soja
order                              
Arima (1, 1, 0)  0.028461   0.03379
Arima (1, 1, 1)  0.030804   0.03379
Arima (2, 1, 0)  0.029644   0.03379
Arima (2, 1, 1)  0.030798   0.03379
Var (1)          0.027753   0.03379
Var (2)          0.029689   0.03379
Var (3)          0.029984   0.03379


Desse modo, para o intervalo de -1,5% e 0,04% de variação diária o Arima (2,1,1) apresentou o menor *MAPE* e, por consequência, o melhor desempenho. Já para o intervalo 0,04% e 2,3%, o melhor modelo foi o Var (1). Para o intervalo de 2,3% e 4,3% o melhor também foi o Var (1). A partir desses resultados destaca-se que o conceito de multi-modelo e multi-ordem podem ser uma alternativa as tradicionais técnicas de modelagem de séries temporais. O exemplo exposto a cima demonstrou que cada intervalo possui um modelo 'ideal'. Ademais, se nos atermos ao resultado do modelo arima, o melhor modelo para o primeiro intervalo seria o Arima (2,1,1), enquanto o modelo ideal para o segundo e terceiro seriam os modelos Arima (1,1,1) e Arima(1,1,0), respectivamente.


## Comparação com as metodologias tradicionais de séries temporais

(Em breve...)